In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

class Adaboost:
    def __init__(self, T):
        self.T = T # Số vòng lặp
        self.alphas = [] # Amount of say của các weak learner
        self.classifiers = [] # Các weak learner

    def train_weak_classifier(self, X, y, w):
        # A simple decision tree
        tree = DecisionTreeClassifier(max_depth=1)
        tree.fit(X, y, sample_weight=w)
        predictions = tree.predict(X)

        # Total error of the weak classifier
        total_error = np.sum(w[y != predictions])

        # Alpha coefficient
        alpha = np.log((1 - total_error) / total_error)

        return predictions, alpha

    def update_weights(self, w, X, y, h, alpha):
        # Update the data weights
        '''
        y: vector nhãn
        h: vector dự đoán
        y*h = 1 hoặc -1
        '''
        new_w = w * np.exp(-alpha * y * h)
        new_w /= np.sum(new_w)
        return new_w

    def fit(self, X, y):
        n = len(X)
        w = np.full(n, 1 / n)  # Initialize data weights

        for i in range(self.T):
            # Create a weak classifier
            h, alpha = self.train_weak_classifier(X, y, w)

            # Update data weights
            w = self.update_weights(w, X, y, h, alpha)

            # Store the weak classifier and its corresponding alpha coefficient
            self.classifiers.append(h)
            self.alphas.append(alpha)

    def predict(self, X):
        # Calculate the predictions of the strong classifier
        predictions = np.zeros(len(X))
        for i in range(len(self.classifiers)):
            h = self.classifiers[i]
            alpha = self.alphas[i]
            predictions += alpha * h.predict(X)
        return np.sign(predictions)
